In [0]:
# Databricks notebook source
# -----------------------------
# STEP 0: Absolute ABFS paths
raw_path        = "abfss://datalake@etldatalakeabhi.dfs.core.windows.net/raw/"
bronze_path     = "abfss://datalake@etldatalakeabhi.dfs.core.windows.net/bronze/"
checkpoint_path = "abfss://datalake@etldatalakeabhi.dfs.core.windows.net/checkpoint/"

# Optional: Test access
display(dbutils.fs.ls(raw_path))


In [0]:
# Databricks notebook source
# -----------------------------
# STEP 1: Read RAW CSV from GitHub (or RAW folder)
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp

github_url = "https://raw.githubusercontent.com/abhishektripathi27/databricks-etl-pipeline/main/data/orders.csv"
pdf = pd.read_csv(github_url)

bronze_df = SparkSession.builder.getOrCreate().createDataFrame(pdf)
bronze_df = bronze_df.withColumn("ingestion_timestamp", current_timestamp())
display(bronze_df)


In [0]:
# Databricks notebook source
# -----------------------------
# STEP 2: Separate good vs bad rows
from pyspark.sql.functions import col, when

# Conditional validation
valid_condition = (
    (col("order_id").isNotNull()) &
    (col("status").isNotNull()) &
    ((col("amount").isNotNull()) | (col("status") == "cancelled"))
)

# Good rows → Bronze
good_bronze = bronze_df.filter(valid_condition)

good_bronze.show()

# Bad rows → RAW quarantine
bad_rows = bronze_df.filter(~valid_condition) \
    .withColumn("bad_data_timestamp", current_timestamp()) \
    .withColumn(
        "error_reason",
        when(col("order_id").isNull(), "Missing order_id")
        .when(col("status").isNull(), "Missing status")
        .when((col("amount").isNull()) & (col("status") != "cancelled"), "Amount missing for non-cancelled order")
        .otherwise("Unknown error")
    )

bad_rows.show()

In [0]:
# Databricks notebook source
# -----------------------------
# STEP 3: Write bad rows to RAW quarantine folder
bad_rows.write.format("delta").mode("append").save(raw_path + "bad_orders/")

spark.sql("""
CREATE TABLE IF NOT EXISTS ws_databricks_etl.bronze.bad_orders
USING DELTA
LOCATION 'abfss://datalake@etldatalakeabhi.dfs.core.windows.net/raw/bad_orders/'
""")


Optional: Register in Unity Catalog

In [0]:
# Databricks notebook source
# -----------------------------
# STEP 4: Upsert good rows into Bronze Delta
from delta.tables import DeltaTable

bronze_table_path = bronze_path + "orders/"

# Create table if doesn't exist
if not DeltaTable.isDeltaTable(spark, bronze_table_path):
    good_bronze.write.format("delta").mode("overwrite").save(bronze_table_path)

# Merge / upsert
bronze_table = DeltaTable.forPath(spark, bronze_table_path)
bronze_table.alias("bronze").merge(
    good_bronze.alias("raw"),
    "bronze.order_id = raw.order_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

# Register table in Unity Catalog
spark.sql(f"""
CREATE TABLE IF NOT EXISTS ws_databricks_etl.bronze.orders
USING DELTA
LOCATION '{bronze_table_path}'
""")


In [0]:
from pyspark.sql.functions import current_timestamp

# Read CSV from raw folder
orders_df = (
    spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(raw_path + "orders/orders.csv")
    .withColumn("ingestion_timestamp", current_timestamp())
)

# Show first few rows
orders_df.display()

# Write to Bronze Delta
orders_df.write.format("delta").mode("overwrite").save(bronze_path + "orders")

# Register in Unity Catalog
spark.sql(f"""
CREATE TABLE IF NOT EXISTS ws_databricks_etl.default.bronze_orders
USING DELTA
LOCATION '{bronze_path}orders'
""")

print("Bronze Orders table created successfully!")


In [0]:
%sql
-- select * from ws_databricks_etl.bronze.bad_orders
select * from ws_databricks_etl.bronze.orders